## DEEP CONVOLUTIONAL GENERATIVE ADVERSERIAL NETWORK

#### The Discriminator and Generator

The purpose of the generator in a GAN is to learn to generate synthetic data that resembles real data samples. It does so by transforming the random noise vectors into meaningful output data, such as images. Through the adversarial training process, where it competes against the discriminator, the generator learns to produce outputs that are indistinguishable from real data to the discriminator.

The discriminator, on the other hand, is responsible for distinguishing between real and fake data samples. It receives both real and fake data samples as input during training and learns to differentiate between them.

By training the generator to fool the discriminator into classifying its generated samples as real, the generator indirectly learns the distribution of real data. However, it does so without explicitly seeing real data samples during training.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import (
    layers,
    models,
    callbacks,
    losses,
    utils,
    metrics,
    optimizers,
    backend
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from IPython import display
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict, namedtuple
from itertools import product
import time
import json
from IPython.display import display

# 0. Parameters

In [2]:
IMAGE_SIZE = 64
CHANNELS = 1
BATCH_SIZE = 128
Z_DIM = 100
EPOCHS = 300
LOAD_MODEL = False
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.999
LEARNING_RATE = 0.0002
DROPOUT_P = 0.25
NOISE_PARAM = 0.1

# 1. Data

In [3]:
train_data = utils.image_dataset_from_directory(
    "C:/Users/jorda/Documents/GenDL_datasets/lego_bricks_dl_dataset",
    labels = None,
    color_mode = "grayscale",
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 42,
    interpolation = "bilinear",
)

Found 46384 files belonging to 1 classes.


In [4]:
def preprocess(img):
    """Scales the data to be between [-1, 1] range."""
    img = (tf.cast(img, "float32") - 127.5) / 127.5
    return img

train = train_data.map(lambda x: preprocess(x))

# 2. Model Architecture

In [11]:
### The discriminator
discriminator = tf.keras.Sequential([
    layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    layers.Conv2D(filters = 64, kernel_size = 4, strides = 2, padding = "same", use_bias = False, name="Conv2D_1"),
    layers.LeakyReLU(0.2),
    layers.Dropout(0.3),
    layers.Conv2D(filters = 128, kernel_size = 4, strides = 2, padding = "same", use_bias = False),
    layers.BatchNormalization(momentum = 0.9),
    layers.LeakyReLU(0.2),
    layers.Dropout(0.3),
    layers.Conv2D(filters = 256, kernel_size = 4, strides = 2, padding = "same", use_bias = False),
    layers.BatchNormalization(momentum = 0.9),
    layers.LeakyReLU(0.2),
    layers.Dropout(0.3),
    layers.Conv2D(filters = 512, kernel_size = 4, strides = 2, padding="same", use_bias = False),
    layers.BatchNormalization(momentum = 0.9),
    layers.LeakyReLU(0.2),
    layers.Dropout(0.3),
    layers.Conv2D(filters = 1, 
                  kernel_size = 4, 
                  strides = 1, 
                  padding = "valid", 
                  use_bias = False, 
                  activation = 'sigmoid'),
    layers.Flatten()
])

discriminator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1 (Conv2D)           (None, 32, 32, 64)        1024      
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 dropout_16 (Dropout)        (None, 32, 32, 64)        0         
                                                                 
 conv2d_19 (Conv2D)          (None, 16, 16, 128)       131072    
                                                                 
 batch_normalization_12 (Ba  (None, 16, 16, 128)       512       
 tchNormalization)                                               
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                      

In [13]:
### The generator
generator = tf.keras.Sequential([
    layers.Input(shape=(100, 1)),
    layers.Reshape((1, 1, 100)),
    layers.Conv2DTranspose(512, kernel_size=4, strides=1, padding="valid", use_bias = False),
    layers.BatchNormalization(momentum=0.9),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same", use_bias = False),
    layers.BatchNormalization(momentum=0.9),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", use_bias = False),
    layers.BatchNormalization(momentum=0.9),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", use_bias = False),
    layers.BatchNormalization(momentum=0.9),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(
        1,
        kernel_size=4,
        strides=2,
        padding="same",
        use_bias = False,
        activation = 'tanh'
    )
])

generator.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 1, 1, 100)         0         
                                                                 
 conv2d_transpose_5 (Conv2D  (None, 4, 4, 512)         819200    
 Transpose)                                                      
                                                                 
 batch_normalization_19 (Ba  (None, 4, 4, 512)         2048      
 tchNormalization)                                               
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 4, 4, 512)         0         
                                                                 
 conv2d_transpose_6 (Conv2D  (None, 8, 8, 256)         2097152   
 Transpose)                                                      
                                                      

# 3. Compiling the GAN

In [ ]:
class DCGAN(models.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(DCGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer):
        super(DCGAN, self).compile()
        self.loss_fn = losses.BinaryCrossentropy()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_metric = metrics.Mean(name="d_loss")
        self.d_real_acc_metric = metrics.BinaryAccuracy(name="d_real_acc")
        self.d_fake_acc_metric = metrics.BinaryAccuracy(name="d_fake_acc")
        self.d_acc_metric = metrics.BinaryAccuracy(name="d_acc")
        self.g_loss_metric = metrics.Mean(name="g_loss")
        self.g_acc_metric = metrics.BinaryAccuracy(name="g_acc")

    @property
    def metrics(self):
        return [
            self.d_loss_metric,
            self.d_real_acc_metric,
            self.d_fake_acc_metric,
            self.d_acc_metric,
            self.g_loss_metric,
            self.g_acc_metric,
        ]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim)
        )

        # Train the discriminator on fake images
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(
                random_latent_vectors, training=True
            )
            real_predictions = self.discriminator(real_images, training=True)
            fake_predictions = self.discriminator(
                generated_images, training=True
            )

            real_labels = tf.ones_like(real_predictions)
            real_noisy_labels = real_labels + NOISE_PARAM * tf.random.uniform(
                tf.shape(real_predictions)
            )
            fake_labels = tf.zeros_like(fake_predictions)
            fake_noisy_labels = fake_labels - NOISE_PARAM * tf.random.uniform(
                tf.shape(fake_predictions)
            )

            d_real_loss = self.loss_fn(real_noisy_labels, real_predictions)
            d_fake_loss = self.loss_fn(fake_noisy_labels, fake_predictions)
            d_loss = (d_real_loss + d_fake_loss) / 2.0

            g_loss = self.loss_fn(real_labels, fake_predictions)

        gradients_of_discriminator = disc_tape.gradient(
            d_loss, self.discriminator.trainable_variables
        )
        gradients_of_generator = gen_tape.gradient(
            g_loss, self.generator.trainable_variables
        )

        self.d_optimizer.apply_gradients(
            zip(gradients_of_discriminator, discriminator.trainable_variables)
        )
        self.g_optimizer.apply_gradients(
            zip(gradients_of_generator, generator.trainable_variables)
        )

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.d_real_acc_metric.update_state(real_labels, real_predictions)
        self.d_fake_acc_metric.update_state(fake_labels, fake_predictions)
        self.d_acc_metric.update_state(
            [real_labels, fake_labels], [real_predictions, fake_predictions]
        )
        self.g_loss_metric.update_state(g_loss)
        self.g_acc_metric.update_state(real_labels, fake_predictions)

        return {m.name: m.result() for m in self.metrics}

In [ ]:
# Create a DCGAN
dcgan = DCGAN(
    discriminator=discriminator, generator=generator, latent_dim=100
)

##### Crucial checkpointing
Checkpointing allows me to save a model's weights during training. I can set the checkpoint at the end of an epoch and if the training process is long, I can just save at a checkpoint and resume training from there at a later time. 

In [ ]:
if LOAD_MODEL:
    dcgan.load_weights("./checkpoint/checkpoint.ckpt")

# 4. Training the GAN

In [ ]:
# Compile it with the two optimizers and their user-defined parameters
dcgan.compile(
    d_optimizer=optimizers.Adam(
        learning_rate=0.0002, beta_1 = 0.5, beta_2 = 0.999
    ),
    g_optimizer=optimizers.Adam(
        learning_rate=0.0002, beta_1 = 0.5, beta_2 = 0.999
    ),
)

In [ ]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=1
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

class ImageGenerator(callbacks.Callback):
    def __init__(self, num_img, latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        self.loader = None
        self.tb = None
    
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch: {epoch}")
        if logs.get('loss') is not None:
            print(f"Train Loss: {logs.get('loss')}")
        if logs.get("accuracy") is not None:
            print(f"Train Accuracy: {logs.get('accuracy')}")
        if logs.get("val_loss") is not None:
            print(f"Validation Loss: {logs.get('val_loss')}")
        if logs.get("val_accuracy") is not None:
            print(f"Validation Accuracy: {logs.get('val_accuracy')}")
        print("\n")
        
        random_latent_vectors = tf.random.normal(
            shape=(self.num_img, self.latent_dim)
        )
        generated_images = self.model.generator(random_latent_vectors)
        generated_images = generated_images * 127.5 + 127.5
        generated_images = generated_images.numpy()
        display(
            generated_images,
            save_to="./output/generated_img_%03d.png" % epoch
        )

In [ ]:
dcgan.fit(
    train,
    epochs=EPOCHS,
    callbacks=[
        model_checkpoint_callback,
        tensorboard_callback,
        ImageGenerator(num_img=10, latent_dim=Z_DIM)
    ],
)

In [0]:
# Save the final models
generator.save("./models/generator")
discriminator.save("./models/discriminator")